In [1]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

 Generates association rules for the results found in the projects history last slice.

In [2]:
#df = pd.read_excel("historical_join.xlsx", keep_default_na=False)
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)

len(df)
df.head()

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,ClickHouse,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Activiti,2,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Activiti,3,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Activiti,4,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Activiti,5,0,1,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [3]:
df2 = df.copy()
df2

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,ClickHouse,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Activiti,2,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Activiti,3,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Activiti,4,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Activiti,5,0,1,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,maxwell,6,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3166,maxwell,7,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3167,maxwell,8,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3168,maxwell,9,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping the lines to identify which projects that don't have DB clues

In [4]:
df2 = df2.groupby("PROJECTS").sum()
df2

,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,Redis,...,ClickHouse,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
PROJECTS,,,,,,,,,,,,,,,,,,,,,
Activiti,55,0,6,2,0,4,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
AmazeFileManager,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Android-IMSI-Catcher-Detector,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Anki-Android,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AntennaPod,55,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zeebe,55,0,0,0,0,2,0,0,0,1,...,0,0,0,3,0,0,0,0,0,0
zeppelin,55,0,8,0,0,1,0,0,0,8,...,0,0,0,9,0,0,0,7,0,0
zipkin,55,0,9,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0


Reseting the index column

In [5]:
df2 = df2.reset_index()
df2

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,ClickHouse,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,55,0,6,2,0,4,0,0,0,...,0,0,0,7,0,0,0,0,0,0
1,AmazeFileManager,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Android-IMSI-Catcher-Detector,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Anki-Android,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AntennaPod,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,zeebe,55,0,0,0,0,2,0,0,0,...,0,0,0,3,0,0,0,0,0,0
311,zeppelin,55,0,8,0,0,1,0,0,0,...,0,0,0,9,0,0,0,7,0,0
312,zipkin,55,0,9,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
313,zookeeper,55,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


New dataframe filtering only lines that last version projects

In [6]:
#df10 = df2.mask(df2 > 1, 1)
df10 = df[(df['COMMITS'] == 10)]
df10

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,ClickHouse,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
9,Activiti,10,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
19,airbyte,10,1,1,0,0,1,0,1,0,...,1,1,1,1,0,0,0,0,0,0
29,airsonic,10,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
39,spring-cloud-alibaba,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,nacos,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3129,yacy_search_server,10,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3139,YalpStore,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3149,SmartTubeNext,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3159,zaproxy,10,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues the principal dataframe 

In [7]:
#df2 = df2.iloc[ : , 1:]
#df3[(df3 == 0).all(axis=1)].index
df2 = df2.drop(df2[(df2.iloc[ : , 2:] == 0).all(axis=1)].index)
df2

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,ClickHouse,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,55,0,6,2,0,4,0,0,0,...,0,0,0,7,0,0,0,0,0,0
2,Android-IMSI-Catcher-Detector,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Anki-Android,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Arduino,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,AsciidocFX,55,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,zaproxy,55,0,7,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
310,zeebe,55,0,0,0,0,2,0,0,0,...,0,0,0,3,0,0,0,0,0,0
311,zeppelin,55,0,8,0,0,1,0,0,0,...,0,0,0,9,0,0,0,7,0,0
312,zipkin,55,0,9,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0


Removing projects that don't have DB clues in the last version 

In [8]:
df3 = df[(df['COMMITS'] == 10)]
#df3.drop(["PROJECTS", "COMMITS"], axis=1, inplace=True)
df3 = df3.drop(df3[(df3.iloc[:,2:] == 0).all(axis=1)].index) # deletando as linhas que possuem valor 0 em todas as colunas
df3

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,ClickHouse,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
9,Activiti,10,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
19,airbyte,10,1,1,0,0,1,0,1,0,...,1,1,1,1,0,0,0,0,0,0
29,airsonic,10,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
39,spring-cloud-alibaba,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,nacos,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3079,apps-android-wikipedia,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3089,wildfly,10,0,1,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3129,yacy_search_server,10,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3159,zaproxy,10,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Merging dataframe grouping and dataframe only last version with DB clues

In [9]:
#df3.merge(df4, on=['PROJECTS'], how='outer', suffixes=['', '_'], indicator=True)
df_mer= pd.merge(df2, df3, on= 'PROJECTS', how='outer', suffixes=['', '_'], indicator=True)
df_mer

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,Vertica_,Teradata_,PostgreSQL_ CockroachDB_,PostGIS_,Etcd_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
0,Activiti,55,0,6,2,0,4,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,both
1,Android-IMSI-Catcher-Detector,55,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
2,Anki-Android,55,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
3,Arduino,55,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
4,AsciidocFX,55,0,1,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,zaproxy,55,0,7,10,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
191,zeebe,55,0,0,0,0,2,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,both
192,zeppelin,55,0,8,0,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,both
193,zipkin,55,0,9,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both


Filtering diference in the last version

In [10]:
df5 = df_mer[(df_mer['_merge'].str.startswith('left'))]
df5

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,Vertica_,Teradata_,PostgreSQL_ CockroachDB_,PostGIS_,Etcd_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
4,AsciidocFX,55,0,1,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6,BungeeCord,55,0,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
14,MPS,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
15,NetGuard,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
18,OpenTripPlanner,55,0,2,2,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
20,OsmAnd,55,0,5,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
24,SpongeForge,55,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
37,avro,55,0,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
43,bisq,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
45,buck,55,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


Merging dataframe only lines that last version projects and dataframe only lines in the last version and not DB clues

In [11]:
df_mer2= pd.merge(df10, df5.iloc[:,:1], on= 'PROJECTS', how='outer', suffixes=['', '_'], indicator=True)
df_mer2

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,_merge
0,Activiti,10,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,left_only
1,airbyte,10,1,1,0,0,1,0,1,0,...,1,1,1,0,0,0,0,0,0,left_only
2,airsonic,10,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,left_only
3,spring-cloud-alibaba,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
4,nacos,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,yacy_search_server,10,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,left_only
313,YalpStore,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
314,SmartTubeNext,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only
315,zaproxy,10,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,left_only


Filtering only lines in the last version and not DB clues

In [12]:
df6 = df_mer2[(df_mer2['_merge'].str.startswith('bo'))]
df6

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,_merge
11,cyclops,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
16,rocketmq,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
18,zookeeper,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
28,nutch,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
36,avro,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
46,AsciidocFX,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
54,bisq,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
87,dl4j-examples,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
92,che,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
95,vert.x,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both


Concatening dataframes

In [13]:
df_concat2 =  pd.concat([df3, df6]).drop_duplicates()
df_concat2

,PROJECTS,COMMITS,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,...,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,_merge
9,Activiti,10,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,NaN
19,airbyte,10,1,1,0,0,1,0,1,0,...,1,1,1,0,0,0,0,0,0,NaN
29,airsonic,10,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,NaN
39,spring-cloud-alibaba,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
49,nacos,10,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,helios,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
270,spring-cloud-netflix,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
280,swagger-core,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both
283,synthea,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,both


In [14]:
#df3 = df3.iloc[ : , 2:] #desconiderandcoluna commits 
#df3[(df3 == 0).all(axis=1) # filtra as linhas que possuem valor igual a 0 em todas as colunas


In [15]:
#df2 = df2.astype(np.uint8)

In [16]:
df_concat2.drop(["PROJECTS","COMMITS","_merge"], axis=1, inplace=True)
df_concat2

,Microsoft Azure Table Storage,MySQL_Maria DB,HyperSQL,MarkLogic,MS SQL Server_Microsoft Azure SQL Database,FileMaker,Singlestore,Riak KV,Redis,Aerospike,...,ClickHouse,Vertica,Teradata,PostgreSQL_ CockroachDB,PostGIS,Etcd,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
19,1,1,0,0,1,0,1,0,1,0,...,1,1,1,1,0,0,0,0,0,0
29,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
39,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
49,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ASSOCIATION RULES

In [17]:
frq_items = apriori(df_concat2, min_support = 0.021, use_colnames = True)

In [18]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)

In [19]:
# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending =False)
print(rules.head())

                                              antecedents  \
272509  (IBM DB2, Oracle, MySQL_Maria DB, SAP SQL Anyw...   
441945  (PostgreSQL_ CockroachDB, IBM DB2, Oracle, Inf...   
441949  (PostgreSQL_ CockroachDB, Oracle, SAP Adaptive...   
441950  (PostgreSQL_ CockroachDB, Oracle, SAP Adaptive...   
441951  (PostgreSQL_ CockroachDB, Oracle, SAP Adaptive...   

                                              consequents  antecedent support  \
272509  (Informix, SQLite, MS SQL Server_Microsoft Azu...            0.025641   
441945  (SQLite, SAP Adaptive Server, MS SQL Server_Mi...            0.025641   
441949      (Informix, SQLite, SAP SQL Anywhere, IBM DB2)            0.025641   
441950  (Firebird, Informix, MS SQL Server_Microsoft A...            0.025641   
441951  (Firebird, SAP SQL Anywhere, MS SQL Server_Mic...            0.025641   

        consequent support   support  confidence  lift  leverage  conviction  
272509            0.025641  0.025641         1.0  39.0  0.024984   

In [20]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
272509,"(IBM DB2, Oracle, MySQL_Maria DB, SAP SQL Anyw...","(Informix, SQLite, MS SQL Server_Microsoft Azu...",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
441945,"(PostgreSQL_ CockroachDB, IBM DB2, Oracle, Inf...","(SQLite, SAP Adaptive Server, MS SQL Server_Mi...",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
441949,"(PostgreSQL_ CockroachDB, Oracle, SAP Adaptive...","(Informix, SQLite, SAP SQL Anywhere, IBM DB2)",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
441950,"(PostgreSQL_ CockroachDB, Oracle, SAP Adaptive...","(Firebird, Informix, MS SQL Server_Microsoft A...",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
441951,"(PostgreSQL_ CockroachDB, Oracle, SAP Adaptive...","(Firebird, SAP SQL Anywhere, MS SQL Server_Mic...",0.025641,0.025641,0.025641,1.000000,39.000000,0.024984,inf
...,...,...,...,...,...,...,...,...,...
588,"(Redis, MySQL_Maria DB)",(SQLite),0.210256,0.133333,0.030769,0.146341,1.097561,0.002735,1.015238
587,"(SQLite, MySQL_Maria DB)",(Redis),0.097436,0.302564,0.030769,0.315789,1.043711,0.001289,1.019329
590,(Redis),"(SQLite, MySQL_Maria DB)",0.302564,0.097436,0.030769,0.101695,1.043711,0.001289,1.004741
5442,"(SQLite, PostgreSQL_ CockroachDB, MySQL_Maria DB)",(Redis),0.082051,0.302564,0.025641,0.312500,1.032839,0.000815,1.014452


In [21]:
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [22]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]

Showing all filtered rules.

In [23]:
rules_filter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequente_len
253,(Firebird),(Informix),0.030769,0.035897,0.025641,0.833333,23.214286,0.024536,5.784615,1,1
252,(Informix),(Firebird),0.035897,0.030769,0.025641,0.714286,23.214286,0.024536,3.392308,1,1
317,(SAP SQL Anywhere),(SAP Adaptive Server),0.056410,0.051282,0.051282,0.909091,17.727273,0.048389,10.435897,1,1
316,(SAP Adaptive Server),(SAP SQL Anywhere),0.051282,0.056410,0.051282,1.000000,17.727273,0.048389,inf,1,1
271,(SAP Adaptive Server),(Informix),0.051282,0.035897,0.030769,0.600000,16.714286,0.028928,2.410256,1,1
...,...,...,...,...,...,...,...,...,...,...,...
187,(Cassandra),(H2),0.056410,0.338462,0.025641,0.454545,1.342975,0.006548,1.212821,1,1
293,(MongoDB),(SQLite),0.143590,0.133333,0.025641,0.178571,1.339286,0.006496,1.055072,1,1
292,(SQLite),(MongoDB),0.133333,0.143590,0.025641,0.192308,1.339286,0.006496,1.060317,1,1
7,(MySQL_Maria DB),(Google Cloud Datastore),0.435897,0.087179,0.046154,0.105882,1.214533,0.008153,1.020918,1,1


In [24]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'historical_rulesv10.xlsx'    #'seq_patterns/testes/historical_rules_commit10_197reg_021.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)